In [1]:
# Associate words with archetypes/character traits as intermediate layer
# and with influencer as the "last" layer

# Dependencies
import pandas as pd
import dask.dataframe as dd
import numpy as np
from tqdm import tqdm
import copy
import os
import toml
import re
import itertools
from text_cleaner import *
import operator
from collections import Counter
import pickle

def extract_hashtags(post_text):
    HASH_RE = re.compile(r"\#\w+")
    out_list = re.findall(HASH_RE, post_text)
    return out_list

In [2]:
# Load the .csv with archetypes
arch_df = pd.read_csv('archetypes_pl_new.csv', index_col=0)

# Save the order of columns
trait_list = arch_df.columns.tolist()

# Show the table header and column list
print(trait_list)
arch_df.head()

['innocent', 'sage', 'explorer', 'outlaw', 'magician', 'hero', 'lover', 'jester', 'everyman', 'caregiver', 'ruler', 'creator', 'dominant', 'submissive', 'maximalist', 'minimalist', 'inspiring', 'systematic', 'discovering', 'conservative', 'verifying', 'overlooking', 'sharpening', 'harmonic', 'empathic', 'matter_of_fact', 'brave', 'protective', 'generous', 'thrifty', 'favourable', 'balanced', 'sensuality', 'intelligent', 'believe', 'egocentric', 'allocentric']


,innocent,sage,explorer,outlaw,magician,hero,lover,jester,everyman,caregiver,...,protective,generous,thrifty,favourable,balanced,sensuality,intelligent,believe,egocentric,allocentric
id,,,,,,,,,,,,,,,,,,,,,
marek_grodzki,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,...,2.0,3.0,4.0,4.0,3.0,4.0,4.0,3.0,0.0,0.0
vege_style_life,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,4.0,4.0,4.0,3.0,3.0,3.0,2.0,0.0,3.0
oliwka__2007,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,...,2.0,2.0,0.0,3.0,1.0,2.0,4.0,1.0,0.0,3.0
z_przestrzeni_serca,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,3.0,0.0,4.0,4.0,3.0,4.0,4.0,0.0,1.0
zaradne_warsztaty,3.0,0.0,0.0,0.0,3.0,0.0,0.0,2.0,3.0,4.0,...,3.0,4.0,0.0,2.0,2.0,4.0,2.0,3.0,1.0,3.0


In [3]:
# Table preprocessing - replace all NaN with 2 (Unrelated/Don't know class), replace 0-5 values with the ones in range -1.0 - 1.0
arch_df = arch_df.fillna(2)

# Remove duplicated annotations, to exclude conflicting entries
arch_df = arch_df[~arch_df.index.duplicated(keep='first')]

# Print the head of the dataset after modification
arch_df.head()

,innocent,sage,explorer,outlaw,magician,hero,lover,jester,everyman,caregiver,...,protective,generous,thrifty,favourable,balanced,sensuality,intelligent,believe,egocentric,allocentric
id,,,,,,,,,,,,,,,,,,,,,
marek_grodzki,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,...,2.0,3.0,4.0,4.0,3.0,4.0,4.0,3.0,0.0,0.0
vege_style_life,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,4.0,4.0,4.0,3.0,3.0,3.0,2.0,0.0,3.0
oliwka__2007,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,...,2.0,2.0,0.0,3.0,1.0,2.0,4.0,1.0,0.0,3.0
z_przestrzeni_serca,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,3.0,0.0,4.0,4.0,3.0,4.0,4.0,0.0,1.0
zaradne_warsztaty,3.0,0.0,0.0,0.0,3.0,0.0,0.0,2.0,3.0,4.0,...,3.0,4.0,0.0,2.0,2.0,4.0,2.0,3.0,1.0,3.0


In [5]:
# Split dataset into train/test, in 75-25% proportion
train_df = arch_df.sample(frac=0.75, random_state=42)
test_df = arch_df.drop(train_df.index).sample(frac=1.0)

BASE_DIR = "instagram_cleared"

def generate_dataset(dataset_frame):
    posts = []

    # Iterate over whole DataFrame
    for i, row in tqdm(dataset_frame.iterrows()):
        profile_posts = []
        profile_hashtags = []

        # Get all posts per profile
        profile_path = os.path.join(BASE_DIR, i)
        for file in os.listdir(profile_path):
            if not file.endswith(".toml"):
                with open(os.path.join(profile_path, file), "r") as post_f:
                    read_text = post_f.read()
                    profile_posts.append(remove_stopwords(clean_up_text(read_text)))
                    profile_hashtags.append(extract_hashtags(read_text))

        # Merge lists - a single list for a single influencer
        profile_hashtags = list(itertools.chain.from_iterable(profile_hashtags))
        posts.append(list(itertools.chain.from_iterable([profile_posts, [profile_hashtags]])))
    return posts

train_posts = generate_dataset(train_df)
test_posts = generate_dataset(test_df)

514it [00:12, 42.50it/s]
171it [00:04, 39.34it/s]


In [7]:
# Map training usernames to indices
users = list(train_df.index.values)
user_indices = {k: users.index(k) for k in users}

In [ ]:
# AGDS - discrete approach

def merge_dicts(dict_a, dict_b) -> dict:
    out_dict = dict_a
    for k, v in dict_b.items():
        if k in out_dict.keys():
            out_dict[k] += v
        else:
            out_dict[k] = v
    return out_dict

def min_max_normalize(df_to_normalize):
    normalized_df = df_to_normalize
    cols = normalized_df.columns.tolist()
    for col in tqdm(cols):
        normalized_df[col] = (normalized_df[col] - normalized_df[col].min()) / (normalized_df[col].max() - normalized_df[col].min())
    return normalized_df
    

# Iterate over all of the traits/archetypes
word_set = set()
trait_df_list = []
for trait in tqdm(trait_list):
    # Select influencers which have the given archetype annotated
    subset_df = train_df[trait]
    trait_subframe = pd.DataFrame()
    for trait_class in range(5):
        class_df = subset_df.loc[subset_df == trait_class]
        subset_indices = [user_indices[idx] for idx in class_df.index.values]

        # Get all posts for the list of influencers
        f = operator.itemgetter(*subset_indices)
        sublist = list(f(train_posts))

        # Counter to calculate each word occurrences
        trait_total = 0
        out_dict = {}
        for i, post_set in enumerate(sublist):
            trait_ctr = Counter(itertools.chain.from_iterable(post_set))
            trait_total += sum(trait_ctr.values())
            out_dict = merge_dicts(out_dict, trait_ctr)
        out_dict = {k: float(v / trait_total) for k, v in out_dict.items()}
        word_set.update(out_dict.keys())
        trait_ctr = {trait_class: out_dict}
        trait_tmp_df = pd.DataFrame.from_dict(trait_ctr, orient="index")
        trait_subframe = trait_subframe.append(trait_tmp_df)

    # Append the new dataframe
    #word_df = word_df.append(trait_subframe)
    
    trait_subframe = min_max_normalize(trait_subframe)
    trait_df_list.append(trait_subframe)
    
softmax_word_df = pd.concat(trait_df_list, keys=trait_list)

 93%|█████████▎| 175363/187588 [01:22<00:05, 2152.20it/s]

 46%|████▋     | 86909/187588 [00:41<00:50, 2011.13it/s]
IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)


 94%|█████████▎| 175505/187588 [01:18<00:05, 2235.39it/s]
IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)


 30%|██▉       | 55626/187588 [00:25<00:59, 2233.46it/s]
IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
T

In [ ]:
# Check the calculation results
softmax_word_df

In [ ]:
# Fill NaN values with 0
softmax_word_df = softmax_word_df.fillna(0)

softmax_word_df.to_pickle("softmax_normalized_word_trait_array.pickle")

In [11]:
# Create word map for softmax structure
softmax_word_map = softmax_word_df.columns.tolist()

In [12]:
from scipy.special import softmax

def get_trait_dot_product(post_text: str, word_map: list, word_dataframe: pd.DataFrame) -> list:
    # Filter out the text
    filtered_post = remove_stopwords(clean_up_text(post_text))
    filtered_post += extract_hashtags(post_text)
    
    # Create a vector for dot product vector
    post_vector = [0] * len(word_map)
    
    # Calculate word occurrences
    word_ctr = Counter(filtered_post)
    
    for word, freq in word_ctr.items():
        if word in word_map:
            post_vector[word_map.index(word)] = freq
    
    # Calculate dot product for a given text
    word_dot = word_dataframe.dot(post_vector)
    
    out_vec = pd.Series()
    for trait in trait_list:
        out_vec = out_vec.append(pd.Series([np.argmax(softmax(word_dot.loc[trait]))], index=[trait]))
    
    return out_vec

# Trait accuracy - round the results
def natural_round(x: float) -> int:
    out = int(x // 1)
    return out + 1 if (x - out) >= 0.5 else out

def accuracy_per_trait(input_vector: pd.Series, annotated_vector: pd.Series) -> np.array:
    out_array = np.array([0] * 37, dtype=np.int)
    for i in range(len(out_array)):
        if input_vector[i] == annotated_vector[i]:
            out_array[i] = 1
    return out_array

In [13]:
pbar = tqdm(train_df.iterrows())
accuracy = 0

# Out accuracy vector
total_accuracy = np.array([0] * 37, dtype=np.int)

for idx, row in pbar:
    user_text = list(itertools.chain.from_iterable(train_posts[users.index(idx)]))
    user_text = " ".join(user_text)
    sim_output = get_trait_dot_product(user_text, softmax_word_map, softmax_word_df)
    user_accuracy = accuracy_per_trait(sim_output, row)
    total_accuracy += user_accuracy
    pbar.set_description(f"Average accuracy: {round(np.mean(np.divide(total_accuracy, users.index(idx)+1))*100, 2)}")

0it [00:00, ?it/s]<ipython-input-12-4ffef00e153a>:21: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  out_vec = pd.Series()
Average accuracy: 81.73: : 514it [28:45,  3.36s/it]


In [15]:
# Show total accuracy
scaled_train_accuracy = np.divide(total_accuracy, len(train_df))
avg_train_accuracy = np.mean(scaled_train_accuracy)

print("--- ACCURACY ON TRAINING DATASET ---")

print(f"Average train dataset accuracy: {round(avg_train_accuracy*100, 2)}%")
print("Accuracy per trait:")
for i in range(len(trait_list)):
    print(f"{trait_list[i]}: {round(scaled_train_accuracy[i] * 100, 2)}%")

--- ACCURACY ON TRAINING DATASET ---
Average train dataset accuracy: 81.73%
Accuracy per trait:
innocent: 79.38%
sage: 77.43%
explorer: 81.71%
outlaw: 84.82%
magician: 94.16%
hero: 82.49%
lover: 90.27%
jester: 87.35%
everyman: 78.02%
caregiver: 89.11%
ruler: 80.93%
creator: 79.77%
dominant: 69.65%
submissive: 86.19%
maximalist: 68.48%
minimalist: 77.04%
inspiring: 72.76%
systematic: 75.49%
discovering: 71.79%
conservative: 90.86%
verifying: 60.31%
overlooking: 76.46%
sharpening: 89.69%
harmonic: 76.85%
empathic: 85.6%
matter_of_fact: 80.54%
brave: 90.47%
protective: 85.8%
generous: 77.63%
thrifty: 89.3%
favourable: 91.44%
balanced: 79.77%
sensuality: 82.1%
intelligent: 72.76%
believe: 87.74%
egocentric: 93.58%
allocentric: 86.19%


In [16]:
# Set up environment for test dataset
test_users = list(test_df.index.values)
test_user_indices = {k: test_users.index(k) for k in test_users}

In [18]:
pbar = tqdm(test_df.iterrows())
accuracy = 0

# Out accuracy vector
test_total_accuracy = np.array([0] * 37, dtype=np.int)

for idx, row in pbar:
    user_text = list(itertools.chain.from_iterable(test_posts[test_users.index(idx)]))
    user_text = " ".join(user_text)
    sim_output = get_trait_dot_product(user_text, softmax_word_map, softmax_word_df)
    user_accuracy = accuracy_per_trait(sim_output, row)
    test_total_accuracy += user_accuracy
    pbar.set_description(f"Average accuracy: {round(np.mean(np.divide(test_total_accuracy, test_users.index(idx)+1))*100, 2)}")

0it [00:00, ?it/s]<ipython-input-12-4ffef00e153a>:21: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  out_vec = pd.Series()
Average accuracy: 24.97: : 171it [10:59,  3.86s/it]


In [19]:
# Show total accuracy
scaled_test_accuracy = np.divide(test_total_accuracy, len(test_df))
avg_test_accuracy = np.mean(scaled_test_accuracy)

print("--- ACCURACY ON TESTING DATASET ---")

print(f"Average test dataset accuracy: {round(avg_test_accuracy*100, 2)}%")
print("Accuracy per trait:")
for i in range(len(trait_list)):
    print(f"{trait_list[i]}: {round(scaled_test_accuracy[i] * 100, 2)}%")

--- ACCURACY ON TESTING DATASET ---
Average test dataset accuracy: 24.97%
Accuracy per trait:
innocent: 29.24%
sage: 23.98%
explorer: 22.81%
outlaw: 30.99%
magician: 29.82%
hero: 30.99%
lover: 27.49%
jester: 23.98%
everyman: 33.92%
caregiver: 18.13%
ruler: 23.39%
creator: 20.47%
dominant: 25.15%
submissive: 27.49%
maximalist: 18.71%
minimalist: 18.13%
inspiring: 19.3%
systematic: 21.64%
discovering: 25.73%
conservative: 25.73%
verifying: 19.3%
overlooking: 20.47%
sharpening: 26.9%
harmonic: 32.16%
empathic: 24.56%
matter_of_fact: 27.49%
brave: 31.58%
protective: 15.79%
generous: 28.07%
thrifty: 27.49%
favourable: 16.96%
balanced: 22.22%
sensuality: 25.73%
intelligent: 25.73%
believe: 27.49%
egocentric: 33.33%
allocentric: 21.64%
